In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import tensorflow as tf
import sys
sys.path.append("..")
from src.prestep.datetime_extension import timedate_process
from src.model.produce_rnn_flow import aggregate_time

In [2]:
%%time
#read block data
df=pd.read_csv("../input/blocks.csv",header=0)
df = aggregate_time(df)
df = timedate_process(df)

CPU times: user 17.2 s, sys: 13 s, total: 30.2 s
Wall time: 30.2 s


In [3]:
pd.set_option('display.max_columns', None)
df.head()

,Level,latBlock,lngBlock,month,day,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11,h12,h13,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23,h0_neighbor1,h1_neighbor1,h2_neighbor1,h3_neighbor1,h4_neighbor1,h5_neighbor1,h6_neighbor1,h7_neighbor1,h8_neighbor1,h9_neighbor1,h10_neighbor1,h11_neighbor1,h12_neighbor1,h13_neighbor1,h14_neighbor1,h15_neighbor1,h16_neighbor1,h17_neighbor1,h18_neighbor1,h19_neighbor1,h20_neighbor1,h21_neighbor1,h22_neighbor1,h23_neighbor1,h0_neighbor2,h1_neighbor2,h2_neighbor2,h3_neighbor2,h4_neighbor2,h5_neighbor2,h6_neighbor2,h7_neighbor2,h8_neighbor2,h9_neighbor2,h10_neighbor2,h11_neighbor2,h12_neighbor2,h13_neighbor2,h14_neighbor2,h15_neighbor2,h16_neighbor2,h17_neighbor2,h18_neighbor2,h19_neighbor2,h20_neighbor2,h21_neighbor2,h22_neighbor2,h23_neighbor2,days,dayofweek,holiday,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (∞C),Maximum Temperature (∞C),Minimum Temperature (∞C),Mean Wind Speed (km/h),Max Wind Speed (km/h),eventFest,eventArt,eventinACM,eventinPSCS
0,0,0,20,4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,4.0,8.5,3.5,6.0,4.0,6.5,13.0,5.0,2.5,1.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,10.5,31.0,5.0,23.0,32.0,24.5,80.0,20.0,3.0,1.0,1.0,1.0,1.0,0.0,0,6,False,8.8,5.8,7.8,8.6,27.7,32.9,25.6,7.2,22.3,False,False,False,False
1,0,0,20,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.5,6.5,7.0,6.5,10.0,7.0,6.5,9.5,1.0,1.0,0.5,1.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,14.5,27.5,29.0,33.0,16.0,16.5,35.5,2.5,1.5,1.5,2.0,1.5,0.5,1,0,False,1.6,1.6,1.6,1.6,28.7,32.8,26.5,8.3,29.5,False,False,False,False
2,0,0,20,4,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,2.0,0.0,1.0,0.5,0.0,0.0,0.0,2.5,1.5,2.0,2.5,1.5,4.0,4.5,3.5,5.0,3.5,3.5,2.5,1.0,1.5,1.5,1.5,0.5,2.0,0.5,2.0,1.0,0.0,0.0,0.5,2.0,1.0,2.0,2.5,4.5,13.5,8.0,10.5,34.0,4.5,7.5,8.5,1.5,1.5,1.5,1.5,0.5,2,1,False,0.0,0.0,0.0,0.0,28.9,33.0,26.8,7.2,25.9,False,False,False,False
3,0,0,20,4,5,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,4.5,10.0,6.0,5.5,5.5,14.0,0.0,3.0,9.0,6.0,5.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,2.0,5.5,3.5,8.0,17.5,10.0,6.5,30.5,29.5,0.0,4.5,44.5,21.5,14.5,31.5,4,3,False,0.0,0.0,0.0,0.0,29.2,33.1,26.5,6.8,24.5,False,False,False,False
4,0,0,20,4,6,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.5,11.0,10.0,5.0,11.5,7.0,7.0,4.0,2.5,3.0,4.5,2.0,1.0,0.5,6.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,3.0,25.0,25.0,20.5,13.0,30.0,16.5,28.5,9.0,12.5,5.5,9.0,1.5,1.5,0.5,5,4,False,0.0,0.0,0.0,0.0,29.2,33.6,26.9,6.8,23.0,False,False,False,False


In [4]:
%%time
for h in range(24):
    df["h%s"%(h)]=np.nan
    df.loc[df.hour==h,"h%s"%(h)] = df.loc[df.hour==h,"ClientMacAddr"]

CPU times: user 4.92 s, sys: 3.71 s, total: 8.63 s
Wall time: 8.65 s


In [5]:
df = df.groupby(["Level","latBlock","lngBlock","month","day"]).\
agg({"h%s"%(h):"max" for h in range(24)}).reset_index().fillna(0).astype("int")

In [6]:
df.head()

,Level,latBlock,lngBlock,month,day,h0,h1,h2,h3,h4,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,0,0,20,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,20,4,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,20,4,3,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
3,0,0,20,4,5,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,20,4,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df1 = deepcopy(df)
df1.loc[:,"lngBlock"]-=1
df1.rename(columns={"h%s"%(h):"h%s_right"%(h) for h in range(24)}, inplace=True)
df2 = deepcopy(df)
df2.loc[:,"lngBlock"]+=1
df2.rename(columns={"h%s"%(h):"h%s_left"%(h) for h in range(24)}, inplace=True)
df3 = deepcopy(df)
df3.loc[:,"latBlock"]-=1
df3.rename(columns={"h%s"%(h):"h%s_upper"%(h) for h in range(24)}, inplace=True)
df4 = deepcopy(df)
df4.loc[:,"latBlock"]+=1
df4.rename(columns={"h%s"%(h):"h%s_lower"%(h) for h in range(24)}, inplace=True)

In [44]:
df = df.merge(df1,how="left",on=["Level","latBlock","lngBlock","month","day"])\
.merge(df2,how="left",on=["Level","latBlock","lngBlock","month","day"])\
.merge(df3,how="left",on=["Level","latBlock","lngBlock","month","day"])\
.merge(df4,how="left",on=["Level","latBlock","lngBlock","month","day"])

In [45]:
for h in range(24):
    df["h%s_neighbor1"%(h)] = df[["h%s_right"%(h),"h%s_left"%(h),"h%s_upper"%(h),"h%s_lower"%(h)]].mean(axis=1)
    df.drop(["h%s_right"%(h),"h%s_left"%(h),"h%s_upper"%(h),"h%s_lower"%(h)],axis=1, inplace=True)

In [ ]:
%%time
df1 = deepcopy(df)
for i in [-1,0,1]:
    for j in [-1,0,1]:
        if i==0 and j==0:
            continue
        df_temp = deepcopy(df1)
        df_temp.loc[:,"lngBlock"]+=i
        df_temp.loc[:,"latBlock"]+=j
        df_temp.rename(columns={"h%s"%(h):"h%s_%s_%s"%(h,i,j) for h in range(24)}, inplace=True)
        df = df.merge(df_temp,how="left",on=["Level","latBlock","lngBlock","month","day"])
        
for h in range(24):
    df["h%s_neighbor1"%(h)] = df[["h%s_%s_%s"%(h,i,j) for i,j in [(0,1),(0,-1),(-1,0),(1,0)]]].mean(axis=1)
    
for h in range(24):
    df["h%s_neighbor2"%(h)] = df[["h%s_%s_%s"%(h,i,j) for i in [-1,1] for j in [-1,1]]].mean(axis=1)
    
df.drop(["h%s_%s_%s"%(h,i,j) for h in range(24) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0],axis=1, inplace=True)

In [16]:
df.head()

,Level,latBlock,lngBlock,month,day,h0,h1,h2,h3,h4,...,h14_neighbor2,h15_neighbor2,h16_neighbor2,h17_neighbor2,h18_neighbor2,h19_neighbor2,h20_neighbor2,h21_neighbor2,h22_neighbor2,h23_neighbor2
0,0,0,20,4,1,0,0,0,0,0,...,32.0,24.5,80.0,20.0,3.0,1.0,1.0,1.0,1.0,0.0
1,0,0,20,4,2,0,0,0,0,0,...,33.0,16.0,16.5,35.5,2.5,1.5,1.5,2.0,1.5,0.5
2,0,0,20,4,3,0,0,0,0,0,...,10.5,34.0,4.5,7.5,8.5,1.5,1.5,1.5,1.5,0.5
3,0,0,20,4,5,0,0,0,0,0,...,10.0,6.5,30.5,29.5,0.0,4.5,44.5,21.5,14.5,31.5
4,0,0,20,4,6,0,0,0,0,0,...,30.0,16.5,28.5,9.0,12.5,5.5,9.0,1.5,1.5,0.5
